# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [8]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [9]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [10]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [11]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_lumpy,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 8.920368357866666 minutes


In [12]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOUSEHOLD_1_004_CA_1_validation,0.0,1.800851,0.1,0.1
1,HOUSEHOLD_1_004_CA_1_validation,1.0,1.620766,0.1,0.1
2,HOUSEHOLD_1_004_CA_1_validation,1.0,1.617841,0.1,0.1
3,HOUSEHOLD_1_004_CA_1_validation,2.0,1.603970,0.1,0.1
4,HOUSEHOLD_1_004_CA_1_validation,2.0,1.653714,0.1,0.1
...,...,...,...,...,...
800194,HOUSEHOLD_2_505_CA_4_validation,4.0,0.908190,0.9,0.9
800195,HOUSEHOLD_2_505_CA_4_validation,0.0,3.663903,0.9,0.9
800196,HOUSEHOLD_2_505_CA_4_validation,0.0,0.366390,0.9,0.9
800197,HOUSEHOLD_2_505_CA_4_validation,0.0,0.036639,0.9,0.9


In [13]:
pattern_df_result_all_params.to_csv('CrostonTSB_Lumpy_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [14]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [15]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.6328688006666672 minutes


In [16]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_004_CA_1_validation,0.1,0.1,0.818144,85.5368,93.0307,55.5122,1.340846,1.790839
1,HOUSEHOLD_1_004_CA_1_validation,0.1,0.2,0.811508,84.8430,91.7426,55.2944,1.329971,1.796273
2,HOUSEHOLD_1_004_CA_1_validation,0.1,0.3,0.813558,85.0573,92.5397,56.5894,1.333331,1.799735
3,HOUSEHOLD_1_004_CA_1_validation,0.1,0.4,0.820859,85.8207,94.6412,58.9892,1.345297,1.801813
4,HOUSEHOLD_1_004_CA_1_validation,0.1,0.5,0.825557,86.3118,96.9376,61.1895,1.352996,1.802548
...,...,...,...,...,...,...,...,...,...
21622,HOUSEHOLD_2_512_CA_2_validation,0.9,0.5,1.031523,103.8541,117.4962,76.6997,1.375365,1.972316
21623,HOUSEHOLD_2_512_CA_2_validation,0.9,0.6,1.014300,102.1200,117.2394,75.2205,1.352400,1.981981
21624,HOUSEHOLD_2_512_CA_2_validation,0.9,0.7,0.993689,100.0449,116.7467,73.1663,1.324919,1.986976
21625,HOUSEHOLD_2_512_CA_2_validation,0.9,0.8,0.974877,98.1509,116.1164,71.3452,1.299836,1.990943


# Check what products has unexpected metrics result

In [17]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_012_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOUSEHOLD_1_012_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOUSEHOLD_1_012_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOUSEHOLD_1_012_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOUSEHOLD_1_012_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
21298,HOUSEHOLD_2_499_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
21299,HOUSEHOLD_2_499_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
21300,HOUSEHOLD_2_499_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
21301,HOUSEHOLD_2_499_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [18]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_012_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOUSEHOLD_1_012_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOUSEHOLD_1_012_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOUSEHOLD_1_012_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOUSEHOLD_1_012_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
21298,HOUSEHOLD_2_499_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
21299,HOUSEHOLD_2_499_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
21300,HOUSEHOLD_2_499_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
21301,HOUSEHOLD_2_499_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [19]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_012_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOUSEHOLD_1_012_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOUSEHOLD_1_012_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOUSEHOLD_1_012_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOUSEHOLD_1_012_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
21298,HOUSEHOLD_2_499_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
21299,HOUSEHOLD_2_499_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
21300,HOUSEHOLD_2_499_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
21301,HOUSEHOLD_2_499_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_012_CA_3_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
163,HOUSEHOLD_1_012_CA_3_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
164,HOUSEHOLD_1_012_CA_3_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
165,HOUSEHOLD_1_012_CA_3_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
166,HOUSEHOLD_1_012_CA_3_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
21298,HOUSEHOLD_2_499_CA_3_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
21299,HOUSEHOLD_2_499_CA_3_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
21300,HOUSEHOLD_2_499_CA_3_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
21301,HOUSEHOLD_2_499_CA_3_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [23]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [24]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 19.850187265917604 %
Percentage of unexpected values of WMAPE is: 19.850187265917604 %
Percentage of unexpected values of SMAPE is: 18.726591760299627 %
Percentage of unexpected values of MAPE is: 19.850187265917604 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [25]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [26]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE       WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                 
0.1   0.1   0.964769  125.342669  133.273299  62.264170  1.253599  1.720006
      0.2   0.928268  121.573671  134.400288  64.305295  1.248944  1.727997
      0.3   0.920443  120.900018  135.703633  66.497780  1.253615  1.743746
      0.4   0.919545  120.969634  137.133869  68.692720  1.261109  1.762675
      0.5   0.922294  121.445747  138.761896  71.083988  1.270826  1.784374
...              ...         ...         ...        ...       ...       ...
0.9   0.5   0.972708  127.157605  143.738914  78.748004  1.417941  2.040096
      0.6   0.976481  127.651479  145.198298  80.785947  1.425771  2.068696
      0.7   0.982483  128.412199  146.943799  83.101633  1.437382  2.101700
      0.8   0.990712  129.439856  149.113897  85.789004  1.452785  2.139866
      0.9   1.000989  130.701692  151.808421  88.786256  1.471167  2.184200

[81 rows x 6 columns]

In [27]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.4)
The optimum Alpha and Beta based on Mean WMAPE is: (0.1, 0.3)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [28]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE      WMAPE      SMAPE      MAPE       MAE      RMSE
Alpha Beta                                                              
0.1   0.1   0.861807  107.60400  128.25870  58.50205  1.064124  1.464232
      0.2   0.861245  107.20090  130.01485  58.77120  1.084223  1.451743
      0.3   0.860109  108.97315  132.48435  60.53060  1.105418  1.459079
      0.4   0.870685  109.33065  135.60360  63.61195  1.097342  1.482429
      0.5   0.878204  109.15975  138.35800  66.70600  1.110543  1.495451
...              ...        ...        ...       ...       ...       ...
0.9   0.5   0.954171  118.76975  144.48650  74.62070  1.238177  1.741862
      0.6   0.958958  118.77270  147.70800  76.68795  1.264880  1.785952
      0.7   0.964512  119.47400  150.81705  79.69275  1.265173  1.820263
      0.8   0.970983  120.79175  153.46440  82.02380  1.275622  1.854372
      0.9   0.974358  122.21185  157.93550  85.25695  1.289714  1.894567

[81 rows x 6 columns]

In [29]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.3)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.2)


# Run Croston model after decide best paramaters

In [30]:
best_alpha = 0.1
best_beta = 0.1

In [31]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [32]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_lumpy, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.041296897350000944 minutes


In [33]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [34]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_004_CA_1_validation,0.806327,104.2023,112.1562,40.6341,1.116453,1.292364
1,HOUSEHOLD_1_004_CA_2_validation,inf,inf,200.0000,NaN,0.641270,0.694817
2,HOUSEHOLD_1_012_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOUSEHOLD_1_032_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
4,HOUSEHOLD_1_032_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
262,HOUSEHOLD_2_499_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
263,HOUSEHOLD_2_505_CA_4_validation,1.281757,184.0471,182.3526,73.4938,0.394387,0.441163
264,HOUSEHOLD_2_506_CA_2_validation,1.720557,185.2908,193.4319,86.6552,0.264701,0.368573
265,HOUSEHOLD_2_510_CA_2_validation,0.794806,79.0103,98.8243,40.8971,1.467334,2.101611


In [35]:
df_result_final.to_csv('CrostonTSB_Lumpy_Test_Data.csv')